In [1]:
!pip install simpletransformers

     |████████████████████████████████| 213 kB 866 kB/s eta 0:00:01
     |████████████████████████████████| 192 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 7.0 MB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 30.8 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 74 kB 2.2 MB/s  eta 0:00:01
     |████████████████████████████████| 4.6 MB 29.8 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 39.1 MB/s eta 0:00:01


  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=a0b516e4a2cb03968ba46fb913cd199195941461a6054787c594284bf85ad2ed
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.1.1
    Uninstalling ipykernel-5.1.1:
      Successfully uninstalled ipykernel-5.1.1
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.55.1
    Uninstalling tqdm-4.55.1:
      Successfully uninstalled tqdm-4.55.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-git 0.11.0 requires nbdime<2.0.0,>=1.1.0, but you have nbdime 2.1.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

from simpletransformers.classification import ClassificationModel, ClassificationArgs

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import shuffle

from scipy.special import softmax

In [3]:
# Are you using Kaggle or another GPU?
gpu_available = True

In [4]:
# Ensures that tweet and user ids do not appear in scientific notation
pd.options.display.float_format = '{:.0f}'.format

# Train and Test Data

In [5]:
train = pd.read_csv('/kaggle/input/twisty-samples/twisty_train.csv', index_col=0)
train

,confirmed_tweet_ids,user_id,mbti,twitter_text,language
14222,559463518056222720,27621196,ISFJ,@alicedeee Ich könnte der stundenlang zuhören!...,German
14668,473510634081239040,855190290,ENFP,@Moaxi @KatrinaJulie kann ich immer noch sehen...,German
889,377531414130229248,133708993,ENFJ,@GerhardMaier fand ich auch damals. Klappt das...,German
7705,578258496380149760,397366447,ENFP,38 qm für 700 warm ... ich muss verrückt sein ...,German
1585,662601975658344448,1305617798,INTP,@dilettiert Willkommen in unserer Welt. Liebe ...,German
...,...,...,...,...,...
13586,680949466871009280,542772869,INTP,accorsi la deve smettere,Dutch
8513,635612601947394048,169452362,INFP,io vado col finale\nlo faccio\nchiudo alle 4 m...,Dutch
1371,466726191257636864,98672812,ENTJ,L'evoluzione dell'Universo in una simulazione ...,Dutch
4695,561655376605872064,548470145,ESFJ,E dopo essermi addormentata fra le tue braccia...,Dutch


In [6]:
test = pd.read_csv('/kaggle/input/twisty-samples/twisty_test.csv', index_col=0)
test

,confirmed_tweet_ids,user_id,mbti,twitter_text,language
8856,158679079082659840,62117549,ENTP,Wisst ihr was das beste an #ibes ist? Dass sie...,German
15218,597014706436612096,17832573,ENFP,Ich hab Connis 7 Minuten letztes Jahr auch geh...,German
3635,134641950128214000,81568595,ENFP,"@fat_jacK47 ja, aber ich wärs nich :D",German
1065,242712774025809920,3253641,ESTJ,@Wally44 danke. Ist runtergeladen :),German
1588,608464865997549568,3091403585,ISTP,@Patienti_A Schlaf gut,German
...,...,...,...,...,...
13638,642030068894691328,1327151641,ENTP,quel momento in cui stai pedalando tranquillam...,Dutch
18491,260711964781731840,548470145,ESFJ,uomo perfetto????? FA SCHIFOOOOOOOOOOOOOOOOOOO...,Dutch
6374,692631634366038016,201361241,INTJ,"@amerutan non ti allarmare, sto accompagnando ...",Dutch
12934,671954187303919616,164412025,INFJ,- insegnarle l'italiano e avevo tipo otto anni...,Dutch


In [7]:
mbti_num_encoding = {
    'ISTJ':0, 'ISFJ':1, 'INFJ':2, 'INTJ':3,
    'ISTP':4, 'ISFP':5, 'INFP':6, 'INTP':7,
    'ESTP':8, 'ESFP':9, 'ENFP':10, 'ENTP':11,
    'ESTJ':12, 'ESFJ':13, 'ENFJ':14, 'ENTJ':15}

In [8]:
train['labels'] = train['mbti'].apply(lambda x: mbti_num_encoding[x])
test['labels'] = test['mbti'].apply(lambda x: mbti_num_encoding[x])

In [9]:
train = train[['twitter_text', 'labels']].rename(columns={'twitter_text':'text'})
test = test[['twitter_text', 'labels']].rename(columns={'twitter_text':'text'})

# Model and Accuracy Metrics

In [10]:
weights_array = compute_class_weight('balanced', classes=np.arange(16), y=train['labels'])

def BERT_model(train_df, epochs, bert_model='bert-base-multilingual-cased'):
    
    model_args = ClassificationArgs(
        num_train_epochs=epochs,
        #learning_rate = 1e-2,
        
        # Twitter sequence length is less than 64 tokens
        # https://peltarion.com/knowledge-center/documentation/cheat-sheets/bert---text-classification-/-cheat-sheet
        train_batch_size = 64,
        overwrite_output_dir=True
    )
    
    model = ClassificationModel(
        'bert', 
        bert_model, 
        use_cuda=gpu_available, #Set to true if using kaggle GPU
        num_labels=16, 
        weight=list(weights_array), 
        args=model_args
    )
    
    model.train_model(train_df)
    #result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=accuracy_score)
    #print(result['acc'])
    return model

In [11]:
def mbti_accuracy(y_true, y_pred):
    # Measures accuracy for mbti classification across 5 accuracy metrics:
    # one match, two matches, three matches, perfect match, average match
    
    # Average match is number of letters match / 4
    
    # Comparing 'ENFJ' as the true class and 'ENFP' as the predicted class,
    # this function returns...
    # [1, 1, 1, 0, 0.75]
     
    # Get index from one hot encoding of y_true
    # Get index of highest softmax/probability output in y_pred
    # y_true_index = np.argmax(y_true, axis=1)
    # y_pred_index = np.argmax(y_pred, axis=1)
    
    # Use the index to identify the corresponding mbti class
    mbti_num_encoding_list = list(mbti_num_encoding)
    y_true_mbti = [mbti_num_encoding_list[idx] for idx in y_true]
    y_pred_mbti = [mbti_num_encoding_list[idx] for idx in y_pred]
    
    one_match = []
    two_matches = []
    three_matches = []
    perfect_match = []
    
    # Perform mbti accuracy measurements
    sum_num_matches = 0
    for i in np.arange(len(y_true_mbti)):
        num_letter_matches = len(set(y_true_mbti[i]) & set(y_pred_mbti[i]))
        
        # At least 1 letter match
        if num_letter_matches == 1:
            one_match += [True]
            two_matches += [False]
            three_matches += [False]
            perfect_match += [False]
            
        # At least 2 letter matches
        elif num_letter_matches == 2:
            one_match += [True]
            two_matches += [True]
            three_matches += [False]
            perfect_match += [False]
            
        # At least 3 letter matches
        elif num_letter_matches == 3:
            one_match += [True]
            two_matches += [True]
            three_matches += [True]
            perfect_match += [False]
           
        # Perfect match
        else:
            one_match += [True]
            two_matches += [True]
            three_matches += [True]
            perfect_match += [True]
        
    # Average/partial matches
        sum_num_matches += num_letter_matches
    avg_num_matches = sum_num_matches/(len(y_true_mbti)*4)*100
    
    return np.round([np.mean(one_match)*100, 
                     np.mean(two_matches)*100, 
                     np.mean(three_matches)*100, 
                     np.mean(perfect_match)*100, 
                     avg_num_matches], 
                    2)

# Compare Training Size Effects on Accuracy

In [12]:
#results_table_train = pd.DataFrame()
#results_table_test = pd.DataFrame()

# Shuffle the training set so its not pre-sorted by language
# from sklearn.utils import shuffle
train = shuffle(train, random_state=0)

In [13]:
def BERT_accuracy(train_df):
    model = BERT_model(train_df, 
                       epochs=4, 
                       bert_model='bert-base-multilingual-cased')
    
    train_result, train_model_outputs, train_wrong_predictions = model.eval_model(train_df)
    test_result, test_model_outputs, test_wrong_predictions = model.eval_model(test)
    
    train_predictions = np.argmax(softmax(train_model_outputs, axis=1), axis=1)
    test_predictions = np.argmax(softmax(test_model_outputs, axis=1), axis=1)
    
    train_acc = mbti_accuracy(train_df['labels'], train_predictions)
    test_acc = mbti_accuracy(test['labels'], test_predictions)
    
    return (train_acc, test_acc)

In [21]:
# Training size variations
np.arange(0, 80001, 16000)

array([    0, 16000, 32000, 48000, 64000, 80000])

Due to space limitations in kaggle, each training size variation is run individually. The results will be aggregated in another jupyter notebook.

In [15]:
number_samples = 80000
result1 = BERT_accuracy(train.iloc[:number_samples])

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [17]:
result1

(array([100.  ,  91.72,  70.28,  48.47,  75.99]),
 array([100.  ,  84.92,  53.1 ,  25.56,  63.  ]))

In [18]:
df_result = pd.DataFrame(data=result1, 
             columns=['One Match', 'Two Matches', 'Three Matches', 'Perfect Match', 'Average Match'])
df_result['Data'] = ['Train', 'Test']
df_result['Number of Samples'] = number_samples
df_result

,One Match,Two Matches,Three Matches,Perfect Match,Average Match,Data,Number of Samples
0,100,92,70,48,76,Train,80000
1,100,85,53,26,63,Test,80000


In [20]:
df_result.to_csv('bert_train_size_variation4.csv')